# 测试集结果

## 1.qnli

In [1]:
# 微调bert mrpc
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
from transformers import BertTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification
from datasets import load_dataset, load_metric
import torch
from transformers import WEIGHTS_NAME, BertConfig, BertTokenizer
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
}
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']
# 设置模型路径
model_path = "/mnt/zhanyuliang/data/nlp_data/theseus/BertFineTrain/download/qnli/bert-base-uncased-qnli/"
# model_path = 'bert-base-uncased'
# 加载模型和分词器
tokenizer = BertTokenizer.from_pretrained(model_path)

config = config_class.from_pretrained(model_path,
                                          num_labels=2,
                                          finetuning_task='qnli',
                                          cache_dir=None)
config.output_hidden_states = True
model = model_class.from_pretrained(model_path,
                                    from_tf=bool('.ckpt' in model_path),
                                    config=config,
                                    cache_dir=None)
# 准备评估
metric = load_metric("glue", "qnli")
model.eval()

# GPU 支持（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/home/zhanyuliang/miniconda3/envs/theseus/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_2305001/75661060.py:28: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "qnli")


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [2]:
from transformers import glue_processors as processors
from transformers import glue_output_modes as output_modes
from transformers import glue_convert_examples_to_features as convert_examples_to_features
task = 'qnli'
test = True
evaluate = False
strtype = 'test'
model_type = 'bert'
data_dir = "/mnt/zhanyuliang/data/nlp_data/GLUE/QNLI"
max_seq_length = 512
processor = processors[task]()
output_mode = output_modes[task]
cached_features_file = os.path.join(data_dir, 'cached_{}_{}_{}_{}'.format(
    strtype,
    'BertTokenizer',
    str(max_seq_length),
    str(task)))

In [3]:
from transformers import InputExample
if os.path.exists(cached_features_file) :
    features = torch.load(cached_features_file)
else:
    label_list = processor.get_labels()
    if task in ['mnli', 'mnli-mm'] and model_type in ['roberta']:
        # HACK(label indices are swapped in RoBERTa pretrained model)
        label_list[1], label_list[2] = label_list[2], label_list[1]
    examples = processor.get_dev_examples(data_dir) if evaluate else processor.get_train_examples(
        data_dir)
    features = convert_examples_to_features(examples,
                                            tokenizer,
                                            label_list=label_list,
                                            max_length=max_seq_length,
                                            output_mode=output_mode,
                                            pad_on_left=bool(model_type in ['xlnet']),
                                            # pad on the left for xlnet
                                            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                            pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                            )
    if test:
        label_list = processor.get_labels()
        if task in ['mnli', 'mnli-mm'] and model_type in ['roberta']:
            # HACK(label indices are swapped in RoBERTa pretrained model)
            label_list[1], label_list[2] = label_list[2], label_list[1]

        if task != 'sts-b':
            dataset = load_dataset("glue", task)
        elif task =='sts-b':
            dataset = load_dataset("glue", 'stsb')
        if task == 'sts-b' or task == 'rte':
            examples = processor.get_dev_examples(data_dir)
            examples = examples[int(len(examples)/2):]
            features = convert_examples_to_features(examples,
                                                    tokenizer,
                                                    label_list=label_list,
                                                    max_length=max_seq_length,
                                                    output_mode=output_mode,
                                                    pad_on_left=bool(model_type in ['xlnet']),
                                                    # pad on the left for xlnet
                                                    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                    pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                                    )
        elif task == 'qqp':
            examples = processor.get_dev_examples(data_dir)
            examples = examples[int(len(examples)/2):]
            features = convert_examples_to_features(examples,
                                                    tokenizer,
                                                    label_list=label_list,
                                                    max_length=max_seq_length,
                                                    output_mode=output_mode,
                                                    pad_on_left=bool(model_type in ['xlnet']),
                                                    # pad on the left for xlnet
                                                    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                    pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                                    )
        elif task == 'mprc' or task == 'wnli' :
            example = dataset['test']  # 对于测试集
            # 生成 InputExample 对象
            examples = []
            for i in range(len(example['sentence1'])):
                if i == 0:
                    continue
                guid = "%s-%s" % ('test', i)
                text_a = example['sentence1'][i]
                text_b = example['sentence2'][i]
                label = label_list[example['label'][i]]
                examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
            # 转换特征
            features = convert_examples_to_features(examples,
                                            tokenizer,
                                            label_list=label_list,
                                            max_length=max_seq_length,
                                            output_mode=output_mode,
                                            pad_on_left=bool(model_type in ['xlnet']),
                                            # pad on the left for xlnet
                                            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                            pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                            )
        elif task == 'cola':
            examples = processor.get_dev_examples(data_dir)
            examples = examples[int(len(examples)/2):]
            features = convert_examples_to_features(examples,
                                                    tokenizer,
                                                    label_list=label_list,
                                                    max_length=max_seq_length,
                                                    output_mode=output_mode,
                                                    pad_on_left=bool(model_type in ['xlnet']),
                                                    # pad on the left for xlnet
                                                    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                    pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                                    )
        elif task == 'qnli':
            examples = processor.get_dev_examples(data_dir)
            examples = examples[int(len(examples)/2):]
            features = convert_examples_to_features(examples,
                                                    tokenizer,
                                                    label_list=label_list,
                                                    max_length=max_seq_length,
                                                    output_mode=output_mode,
                                                    pad_on_left=bool(model_type in ['xlnet']),
                                                    # pad on the left for xlnet
                                                    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                    pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                                    )

In [4]:
cached_features_file

'/mnt/zhanyuliang/data/nlp_data/GLUE/QNLI/cached_test_BertTokenizer_512_qnli'

In [5]:
len(features)

2732

In [6]:
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)

all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
if output_mode == "classification":
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
elif output_mode == "regression":
    all_labels = torch.tensor([f.label for f in features], dtype=torch.float)

dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)

In [7]:
eval_sampler = SequentialSampler(dataset)
dataset = DataLoader(dataset, sampler=eval_sampler, batch_size=8)

In [8]:
from tqdm import tqdm, trange
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = 'bert'
eval_loss = 0.0
nb_eval_steps = 0
preds = None
for batch in tqdm(dataset, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        inputs = {'input_ids': batch[0],
                    'attention_mask': batch[1],
                    'labels': batch[3]}
        if model_type != 'distilbert':
            inputs['token_type_ids'] = batch[2] if model_type in ['bert',
                                                                        'xlnet'] else None  # XLM, DistilBERT and RoBERTa don't use segment_ids
        outputs = model(**inputs)
        tmp_eval_loss, logits = outputs[:2]

        eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if preds is None:
        preds = logits.detach().cpu().numpy()
        out_label_ids = inputs['labels'].detach().cpu().numpy()
    else:
        preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)


Evaluating: 100%|██████████| 342/342 [00:29<00:00, 11.57it/s]


In [9]:
output_mode = "classification"
eval_loss = eval_loss / nb_eval_steps
if output_mode == "classification":
    preds = np.argmax(preds, axis=1)
elif output_mode == "regression":
    preds = np.squeeze(preds)

In [10]:
task

'qnli'

In [11]:
from transformers import glue_compute_metrics as compute_metrics
result = compute_metrics('qqp', preds, out_label_ids)

In [12]:
result

{'acc': 0.9121522693997072,
 'f1': 0.9118295371050699,
 'acc_and_f1': 0.9119909032523885}

## 2.MNLI

In [5]:
# 微调bert mrpc
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from transformers import BertTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification
from datasets import load_dataset, load_metric
import torch
from transformers import WEIGHTS_NAME, BertConfig, BertTokenizer
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
}
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']
# 设置模型路径
model_path = "/mnt/zhanyuliang/data/nlp_data/theseus/BertFineTrain/download/mnli/bert-base-uncased-MNLI/"
# model_path = 'bert-base-uncased'
# 加载模型和分词器
tokenizer = BertTokenizer.from_pretrained(model_path)

config = config_class.from_pretrained(model_path,
                                          num_labels=3,
                                          finetuning_task='mnli',
                                          cache_dir=None)
config.output_hidden_states = True
model = model_class.from_pretrained(model_path,
                                    from_tf=bool('.ckpt' in model_path),
                                    config=config,
                                    cache_dir=None)
# 准备评估
metric = load_metric("glue", "mnli_mismatched")
model.eval()

# GPU 支持（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [24]:
from transformers import glue_processors as processors
from transformers import glue_output_modes as output_modes
from transformers import glue_convert_examples_to_features as convert_examples_to_features
task = 'mnli'
test = True
evaluate = True
strtype = 'test'
model_type = 'bert'
data_dir = "/mnt/zhanyuliang/data/nlp_data/GLUE/MNLI"
max_seq_length = 128
processor = processors[task]()
output_mode = output_modes[task]
cached_features_file = os.path.join(data_dir, 'cached_{}_{}_{}_{}'.format(
    strtype,
    'BertTokenizer',
    str(max_seq_length),
    str(task)))

In [35]:
from transformers import InputExample
if os.path.exists(cached_features_file) :
    features = torch.load(cached_features_file)
else:
    label_list = processor.get_labels()
    if task in ['mnli', 'mnli-mm'] and model_type in ['roberta']:
        # HACK(label indices are swapped in RoBERTa pretrained model)
        label_list[1], label_list[2] = label_list[2], label_list[1]
    examples = processor.get_dev_examples(data_dir) if evaluate else processor.get_train_examples(
        data_dir)
    features = convert_examples_to_features(examples,
                                            tokenizer,
                                            label_list=label_list,
                                            max_length=max_seq_length,
                                            output_mode=output_mode,
                                            pad_on_left=bool(model_type in ['xlnet']),
                                            # pad on the left for xlnet
                                            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                            pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                            )
    if test:
        label_list = processor.get_labels()
        if task in ['mnli', 'mnli-mm'] and model_type in ['roberta']:
            # HACK(label indices are swapped in RoBERTa pretrained model)
            label_list[1], label_list[2] = label_list[2], label_list[1]

        if task != 'sts-b':
            dataset = load_dataset("glue", task)
        elif task =='sts-b':
            dataset = load_dataset("glue", 'stsb')
        if task == 'sts-b' or task == 'rte':
            examples = processor.get_dev_examples(data_dir)
            examples = examples[int(len(examples)/2):]
            features = convert_examples_to_features(examples,
                                                    tokenizer,
                                                    label_list=label_list,
                                                    max_length=max_seq_length,
                                                    output_mode=output_mode,
                                                    pad_on_left=bool(model_type in ['xlnet']),
                                                    # pad on the left for xlnet
                                                    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                    pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                                    )
        elif task == 'qqp':
            examples = processor.get_dev_examples(data_dir)
            examples = examples[int(len(examples)/2):]
            features = convert_examples_to_features(examples,
                                                    tokenizer,
                                                    label_list=label_list,
                                                    max_length=max_seq_length,
                                                    output_mode=output_mode,
                                                    pad_on_left=bool(model_type in ['xlnet']),
                                                    # pad on the left for xlnet
                                                    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                    pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                                    )
        elif task == 'mprc' or task == 'wnli' :
            example = dataset['test']  # 对于测试集
            # 生成 InputExample 对象
            examples = []
            for i in range(len(example['sentence1'])):
                if i == 0:
                    continue
                guid = "%s-%s" % ('test', i)
                text_a = example['sentence1'][i]
                text_b = example['sentence2'][i]
                label = label_list[example['label'][i]]
                examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
            # 转换特征
            features = convert_examples_to_features(examples,
                                            tokenizer,
                                            label_list=label_list,
                                            max_length=max_seq_length,
                                            output_mode=output_mode,
                                            pad_on_left=bool(model_type in ['xlnet']),
                                            # pad on the left for xlnet
                                            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                            pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                            )
        elif task == 'cola':
            examples = processor.get_dev_examples(data_dir)
            examples = examples[int(len(examples)/2):]
            features = convert_examples_to_features(examples,
                                                    tokenizer,
                                                    label_list=label_list,
                                                    max_length=max_seq_length,
                                                    output_mode=output_mode,
                                                    pad_on_left=bool(model_type in ['xlnet']),
                                                    # pad on the left for xlnet
                                                    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                    pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                                    )
        elif task == 'qnli':
            examples = processor.get_dev_examples(data_dir)
            examples = examples[int(len(examples)/2):]
            features = convert_examples_to_features(examples,
                                                    tokenizer,
                                                    label_list=label_list,
                                                    max_length=max_seq_length,
                                                    output_mode=output_mode,
                                                    pad_on_left=bool(model_type in ['xlnet']),
                                                    # pad on the left for xlnet
                                                    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                    pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                                    )
        elif task in ['mnli', 'mnli-mm']:
            examples = processor.get_dev_examples(data_dir)
            examples = examples[int(len(examples)/2):]
            features = convert_examples_to_features(examples,
                                                    tokenizer,
                                                    label_list=label_list,
                                                    max_length=max_seq_length,
                                                    output_mode=output_mode,
                                                    pad_on_left=bool(model_type in ['xlnet']),
                                                    # pad on the left for xlnet
                                                    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                    pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                                    )
        if 

In [36]:
len(features)

4908

In [7]:
m = load_dataset("glue", 'mnli_mismatched')

In [27]:
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)

all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
if output_mode == "classification":
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
elif output_mode == "regression":
    all_labels = torch.tensor([f.label for f in features], dtype=torch.float)

dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)

In [28]:
eval_sampler = SequentialSampler(dataset)
dataset = DataLoader(dataset, sampler=eval_sampler, batch_size=8)

In [29]:
from tqdm import tqdm, trange
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = 'bert'
eval_loss = 0.0
nb_eval_steps = 0
preds = None
for batch in tqdm(dataset, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        inputs = {'input_ids': batch[0],
                    'attention_mask': batch[1],
                    'labels': batch[3]}
        if model_type != 'distilbert':
            inputs['token_type_ids'] = batch[2] if model_type in ['bert',
                                                                        'xlnet'] else None  # XLM, DistilBERT and RoBERTa don't use segment_ids
        outputs = model(**inputs)
        tmp_eval_loss, logits = outputs[:2]

        eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if preds is None:
        preds = logits.detach().cpu().numpy()
        out_label_ids = inputs['labels'].detach().cpu().numpy()
    else:
        preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)


Evaluating:   0%|                                                                                                                                                                                                                                                                 | 0/614 [00:00<?, ?it/s]

Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 614/614 [00:22<00:00, 27.30it/s]


In [30]:
output_mode = "classification"
eval_loss = eval_loss / nb_eval_steps
if output_mode == "classification":
    preds = np.argmax(preds, axis=1)
elif output_mode == "regression":
    preds = np.squeeze(preds)

In [31]:
task

'mnli'

In [32]:
from transformers import glue_compute_metrics as compute_metrics
result = compute_metrics('mnli', preds, out_label_ids)

In [33]:
result

{'acc': 0.8420945395273024}

## 3.SST-2

In [38]:
# 微调bert mrpc
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from transformers import BertTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification
from datasets import load_dataset, load_metric
import torch
from transformers import WEIGHTS_NAME, BertConfig, BertTokenizer
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
}
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']
# 设置模型路径
model_path = "/mnt/zhanyuliang/data/nlp_data/theseus/BertFineTrain/download/sst2/bert-base-uncased-finetuned-sst2/"
# model_path = 'bert-base-uncased'
# 加载模型和分词器
tokenizer = BertTokenizer.from_pretrained(model_path)

config = config_class.from_pretrained(model_path,
                                          num_labels=2,
                                          finetuning_task='sst2',
                                          cache_dir=None)
config.output_hidden_states = True
model = model_class.from_pretrained(model_path,
                                    from_tf=bool('.ckpt' in model_path),
                                    config=config,
                                    cache_dir=None)
# 准备评估
metric = load_metric("glue", "sst2")
model.eval()

# GPU 支持（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [63]:
from transformers import glue_processors as processors
from transformers import glue_output_modes as output_modes
from transformers import glue_convert_examples_to_features as convert_examples_to_features
task = 'sst-2'
test = True
evaluate = True
strtype = 'test'
model_type = 'bert'
data_dir = "/mnt/zhanyuliang/data/nlp_data/GLUE/SST-2"
max_seq_length = 128
processor = processors[task]()
output_mode = output_modes[task]
cached_features_file = os.path.join(data_dir, 'cached_{}_{}_{}_{}'.format(
    strtype,
    'BertTokenizer',
    str(max_seq_length),
    str(task)))

In [74]:
from transformers import InputExample
if os.path.exists(cached_features_file) :
    features = torch.load(cached_features_file)
else:
    label_list = processor.get_labels()
    if task in ['mnli', 'mnli-mm'] and model_type in ['roberta']:
        # HACK(label indices are swapped in RoBERTa pretrained model)
        label_list[1], label_list[2] = label_list[2], label_list[1]
    examples = processor.get_dev_examples(data_dir) if evaluate else processor.get_train_examples(
        data_dir)
    features = convert_examples_to_features(examples,
                                            tokenizer,
                                            label_list=label_list,
                                            max_length=max_seq_length,
                                            output_mode=output_mode,
                                            pad_on_left=bool(model_type in ['xlnet']),
                                            # pad on the left for xlnet
                                            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                            pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                            )
    if test:
        label_list = processor.get_labels()
        if task in ['mnli', 'mnli-mm'] and model_type in ['roberta']:
            # HACK(label indices are swapped in RoBERTa pretrained model)
            label_list[1], label_list[2] = label_list[2], label_list[1]

        if task != 'sts-b' and task != 'sst-2':
            dataset = load_dataset("glue", task)
        elif task =='sts-b':
            dataset = load_dataset("glue", 'stsb')
        elif task =='sst-2':
            dataset = load_dataset("glue", 'sst2')

        if task == 'sts-b' or task == 'rte' or task == 'sst-2' or task == 'qqp' or task == 'cola' or task == 'qnli' or task in ['mnli', 'mnli-mm']:
            examples = processor.get_dev_examples(data_dir)
            examples = examples[int(len(examples)/2):]
            features = convert_examples_to_features(examples,
                                                    tokenizer,
                                                    label_list=label_list,
                                                    max_length=max_seq_length,
                                                    output_mode=output_mode,
                                                    pad_on_left=bool(model_type in ['xlnet']),
                                                    # pad on the left for xlnet
                                                    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                    pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                                    )
        elif task == 'mprc' or task == 'wnli' :
            example = dataset['test']  # 对于测试集
            # 生成 InputExample 对象
            examples = []
            for i in range(len(example['sentence1'])):
                if i == 0:
                    continue
                guid = "%s-%s" % ('test', i)
                text_a = example['sentence1'][i]
                text_b = example['sentence2'][i]
                label = label_list[example['label'][i]]
                examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
            # 转换特征
            features = convert_examples_to_features(examples,
                                            tokenizer,
                                            label_list=label_list,
                                            max_length=max_seq_length,
                                            output_mode=output_mode,
                                            pad_on_left=bool(model_type in ['xlnet']),
                                            # pad on the left for xlnet
                                            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                            pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                            )

In [75]:
len(features)

436

In [67]:
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)

all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
if output_mode == "classification":
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
elif output_mode == "regression":
    all_labels = torch.tensor([f.label for f in features], dtype=torch.float)

dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)

In [68]:
eval_sampler = SequentialSampler(dataset)
dataset = DataLoader(dataset, sampler=eval_sampler, batch_size=8)

In [69]:
from tqdm import tqdm, trange
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = 'bert'
eval_loss = 0.0
nb_eval_steps = 0
preds = None
for batch in tqdm(dataset, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        inputs = {'input_ids': batch[0],
                    'attention_mask': batch[1],
                    'labels': batch[3]}
        if model_type != 'distilbert':
            inputs['token_type_ids'] = batch[2] if model_type in ['bert',
                                                                        'xlnet'] else None  # XLM, DistilBERT and RoBERTa don't use segment_ids
        outputs = model(**inputs)
        tmp_eval_loss, logits = outputs[:2]

        eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if preds is None:
        preds = logits.detach().cpu().numpy()
        out_label_ids = inputs['labels'].detach().cpu().numpy()
    else:
        preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)


Evaluating:   0%|                                                                                                                                                                                                                                                                  | 0/55 [00:00<?, ?it/s]

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [00:02<00:00, 23.95it/s]


In [70]:
output_mode = "classification"
eval_loss = eval_loss / nb_eval_steps
if output_mode == "classification":
    preds = np.argmax(preds, axis=1)
elif output_mode == "regression":
    preds = np.squeeze(preds)

In [71]:
task

'sst-2'

In [72]:
from transformers import glue_compute_metrics as compute_metrics
result = compute_metrics('sst-2', preds, out_label_ids)

In [73]:
result

{'acc': 0.9197247706422018}

In [2]:
from datasets import load_dataset, load_metric
load_dataset("glue", 'mnli-mm')

/home/zhanyuliang/miniconda3/envs/theseus/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: BuilderConfig 'mnli-mm' not found. Available: ['cola', 'sst2', 'mrpc', 'qqp', 'stsb', 'mnli', 'mnli_mismatched', 'mnli_matched', 'qnli', 'rte', 'wnli', 'ax']

# 3.STS-B

In [13]:
# 微调bert mrpc
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
from transformers import BertTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification
from datasets import load_dataset, load_metric
import torch
from transformers import WEIGHTS_NAME, BertConfig, BertTokenizer
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
}
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']
# 设置模型路径
model_path = "/mnt/zhanyuliang/data/nlp_data/theseus/BertFineTrain/download/stsb/bert-base-uncased-stsb/"
# model_path = 'bert-base-uncased'
# 加载模型和分词器
tokenizer = BertTokenizer.from_pretrained(model_path)

config = config_class.from_pretrained(model_path,
                                          num_labels=1,
                                          finetuning_task='stsb',
                                          cache_dir=None)
config.output_hidden_states = True
model = model_class.from_pretrained(model_path,
                                    from_tf=bool('.ckpt' in model_path),
                                    config=config,
                                    cache_dir=None)
# 准备评估
metric = load_metric("glue", "stsb")
model.eval()

# GPU 支持（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [14]:
from transformers import glue_processors as processors
from transformers import glue_output_modes as output_modes
from transformers import glue_convert_examples_to_features as convert_examples_to_features
task = 'sts-b'
test = False
evaluate = True
strtype = 'dev'
model_type = 'bert'
data_dir = "/mnt/zhanyuliang/data/nlp_data/GLUE/STS-B"
max_seq_length = 128
processor = processors[task]()
output_mode = output_modes[task]
cached_features_file = os.path.join(data_dir, 'cached_{}_{}_{}_{}'.format(
    strtype,
    'BertTokenizer',
    str(max_seq_length),
    str(task)))

In [15]:
from transformers import InputExample
if os.path.exists(cached_features_file) :
    features = torch.load(cached_features_file)
else:
    label_list = processor.get_labels()
    if task in ['mnli', 'mnli-mm'] and model_type in ['roberta']:
        # HACK(label indices are swapped in RoBERTa pretrained model)
        label_list[1], label_list[2] = label_list[2], label_list[1]
    examples = processor.get_dev_examples(data_dir) if evaluate else processor.get_train_examples(
        data_dir)
    features = convert_examples_to_features(examples,
                                            tokenizer,
                                            label_list=label_list,
                                            max_length=max_seq_length,
                                            output_mode=output_mode,
                                            pad_on_left=bool(model_type in ['xlnet']),
                                            # pad on the left for xlnet
                                            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                            pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                            )
    if test:
        label_list = processor.get_labels()
        if task in ['mnli', 'mnli-mm'] and model_type in ['roberta']:
            # HACK(label indices are swapped in RoBERTa pretrained model)
            label_list[1], label_list[2] = label_list[2], label_list[1]

        if task != 'sts-b' and task != 'sst-2':
            dataset = load_dataset("glue", task)
        elif task =='sts-b':
            dataset = load_dataset("glue", 'stsb')
        elif task =='sst-2':
            dataset = load_dataset("glue", 'sst2')

        if task == 'sts-b' or task == 'rte' or task == 'sst-2' or task == 'qqp' or task == 'cola' or task == 'qnli' or task in ['mnli', 'mnli-mm']:
            examples = processor.get_dev_examples(data_dir)
            examples = examples[int(len(examples)/2):]
            features = convert_examples_to_features(examples,
                                                    tokenizer,
                                                    label_list=label_list,
                                                    max_length=max_seq_length,
                                                    output_mode=output_mode,
                                                    pad_on_left=bool(model_type in ['xlnet']),
                                                    # pad on the left for xlnet
                                                    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                    pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                                    )
        elif task == 'mprc' or task == 'wnli' :
            example = dataset['test']  # 对于测试集
            # 生成 InputExample 对象
            examples = []
            for i in range(len(example['sentence1'])):
                if i == 0:
                    continue
                guid = "%s-%s" % ('test', i)
                text_a = example['sentence1'][i]
                text_b = example['sentence2'][i]
                label = label_list[example['label'][i]]
                examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
            # 转换特征
            features = convert_examples_to_features(examples,
                                            tokenizer,
                                            label_list=label_list,
                                            max_length=max_seq_length,
                                            output_mode=output_mode,
                                            pad_on_left=bool(model_type in ['xlnet']),
                                            # pad on the left for xlnet
                                            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                            pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                            )

In [16]:
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)

all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
if output_mode == "classification":
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
elif output_mode == "regression":
    all_labels = torch.tensor([f.label for f in features], dtype=torch.float)

dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)

In [17]:
eval_sampler = SequentialSampler(dataset)
dataset = DataLoader(dataset, sampler=eval_sampler, batch_size=8)

In [18]:
from tqdm import tqdm, trange
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = 'bert'
eval_loss = 0.0
nb_eval_steps = 0
preds = None
for batch in tqdm(dataset, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        inputs = {'input_ids': batch[0],
                    'attention_mask': batch[1],
                    'labels': batch[3]}
        if model_type != 'distilbert':
            inputs['token_type_ids'] = batch[2] if model_type in ['bert',
                                                                        'xlnet'] else None  # XLM, DistilBERT and RoBERTa don't use segment_ids
        outputs = model(**inputs)
        tmp_eval_loss, logits = outputs[:2]

        eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if preds is None:
        preds = logits.detach().cpu().numpy()
        out_label_ids = inputs['labels'].detach().cpu().numpy()
    else:
        preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)


Evaluating: 100%|██████████| 188/188 [00:06<00:00, 30.50it/s]


In [19]:
output_mode = "regression"
eval_loss = eval_loss / nb_eval_steps
if output_mode == "classification":
    preds = np.argmax(preds, axis=1)
elif output_mode == "regression":
    preds = np.squeeze(preds)

In [20]:
task

'sts-b'

In [21]:
from transformers import glue_compute_metrics as compute_metrics
result = compute_metrics('sts-b', preds, out_label_ids)

In [22]:
result

{'pearson': 0.8900648185341595,
 'spearmanr': 0.8872509371354497,
 'corr': 0.8886578778348047}

In [23]:
(0.89+0.887)/2

0.8885000000000001

In [24]:
# 微调bert mrpc
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from transformers import BertTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification
from datasets import load_dataset, load_metric
import torch
from transformers import WEIGHTS_NAME, BertConfig, BertTokenizer
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
}
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']
# 设置模型路径
model_path = "/mnt/zhanyuliang/data/nlp_data/theseus/BertFineTrain/download/mrpc/bert-base-uncased-glue-mrpc/"
# model_path = 'bert-base-uncased'
# 加载模型和分词器
tokenizer = BertTokenizer.from_pretrained(model_path)

config = config_class.from_pretrained(model_path,
                                          num_labels=2,
                                          finetuning_task='mrpc',
                                          cache_dir=None)
config.output_hidden_states = True
model = model_class.from_pretrained(model_path,
                                    from_tf=bool('.ckpt' in model_path),
                                    config=config,
                                    cache_dir=None)
# 准备评估
metric = load_metric("glue", "mrpc")
model.eval()

# GPU 支持（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [1]:
from transformers import AutoTokenizer
model_path = "/mnt/zhanyuliang/data/nlp_data/theseus/BertFineTrain/download/mrpc/bert-base-uncased-glue-mrpc/"

tokenizer = AutoTokenizer.from_pretrained(model_path,        
        cache_dir=None,
        use_fast=True,
        revision=False,
        use_auth_token=False)

/home/zhanyuliang/miniconda3/envs/theseus/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import transformers
print(transformers.__version__)``

2.4.0
